In [1]:
import numpy as np
import pandas as pd
import os
import nltk

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
path = "/content/drive/MyDrive/Capstone/Reviews/All"


In [4]:
os.chdir(path)

In [5]:
lib = os.listdir(path)

In [7]:
#!rm -rf "/content/drive/MyDrive/Capstone/Reviews/All/rel_component"

In [8]:
lines = {}
for i in range(len(lib)):
    with open(lib[i], encoding = 'utf-8') as f:
        lines[i] = f.readlines()

In [62]:
a = pd.DataFrame({'reviews' : [], 'sentences' : []})

In [63]:
for i in range(len(lines)):
    for j in range(len(lines[i])):
        d = {'reviews': i, 'sentences':lines[i][j]}
        a = a.append(d, ignore_index = True)

In [64]:
a

,reviews,sentences
0,0.0,I got the IPhone 8 yesterday.\n
1,0.0,I am really happy with it!\n
2,0.0,It looks and works like new so far.\n
3,1.0,Bought this to replace iPhone 5s.\n
4,1.0,I’m happy I did.\n
5,1.0,Only drawback I have found is has poor recepti...
6,2.0,"Works great, looks like new, so far no issues.\n"
7,2.0,Great price\n
8,3.0,This is the first time I’ve bought a phone not...


In [65]:
pwd

'/content'

In [66]:
%cd '/content'

/content


In [67]:
pwd

'/content'

In [44]:
%%capture
!pip install -U spacy-nightly --pre
!pip install -U pip setuptools wheel
!python -m spacy project clone tutorials/rel_component
!python -m spacy download en_core_web_trf
!pip install -U spacy transformers
!pip install typer
!pip install spacy[transformers]

In [45]:
import spacy
import random
import typer
from pathlib import Path
import spacy
from spacy.tokens import DocBin, Doc
from spacy.training.example import Example
from rel_pipe import make_relation_extractor, score_relations
from rel_model import create_relation_model, create_classification_layer, create_instances, create_tensors

In [68]:
text = a['sentences'].to_list()

In [69]:
len(text)

9

In [70]:
text

['I got the IPhone 8 yesterday.\n',
 'I am really happy with it!\n',
 'It looks and works like new so far.\n',
 'Bought this to replace iPhone 5s.\n',
 'I’m happy I did.\n',
 'Only drawback I have found is has poor reception inside of the machine shop I work in compared to the 5s.\n',
 'Works great, looks like new, so far no issues.\n',
 'Great price\n',
 'This is the first time I’ve bought a phone not from my phone company and it came with a broken camera filled with debris so now I have to send it back and hope I get a fully functioning phone back\n']

In [71]:
nlp = spacy.load("/content/drive/MyDrive/Capstone/content/model-best")
nlp.add_pipe('sentencizer')

In [72]:
text

['I got the IPhone 8 yesterday.\n',
 'I am really happy with it!\n',
 'It looks and works like new so far.\n',
 'Bought this to replace iPhone 5s.\n',
 'I’m happy I did.\n',
 'Only drawback I have found is has poor reception inside of the machine shop I work in compared to the 5s.\n',
 'Works great, looks like new, so far no issues.\n',
 'Great price\n',
 'This is the first time I’ve bought a phone not from my phone company and it came with a broken camera filled with debris so now I have to send it back and hope I get a fully functioning phone back\n']

In [88]:
b = pd.DataFrame({'sentences':[], 'ASP':[], 'OPI':[] })

In [89]:
b

,sentences,ASP,OPI


In [87]:
#for doc in nlp.pipe(text, disable=["tagger"]):
for e in doc.ents:
  print(e)

broken
camera
debris
fully functioning


In [76]:
if doc.ents:
  print('True')
else:
  print('False')

True


In [77]:
for e in doc.ents:
  print (e.label_)

OPI
ASP
OPI
OPI


In [78]:
#please ignore this cell this has been taken as a trial. actual code is in the next cell
for doc in nlp.pipe(text, disable=["tagger"]):
#  print("The doc is: ", doc)

  if doc.ents:
    k = []
    for e in doc.ents:
      k.append(e.label_)
      #print('doc ents :', e)
      #print(e.start, e.text, e.label_)
      for i in k:
        if (i == 'OPI'):
          e1 = e.text
          e2 = []
        elif  (i == 'ASP'):
          e2 = e.text
          e1= []
      d1 = {'sentences': str(doc) , 'ASP':e2, 'OPI':e1}
      b = b.append(d1, ignore_index = True)

  else:
      d1 = {'sentences': str(doc) , 'ASP': [], 'OPI': []}
      b = b.append(d1, ignore_index = True)

        #a['OPI'] = e.text
 # print(f"spans: {[(e.start, e.text, e.label_) for e in doc.ents]}")
   

In [90]:
for doc in nlp.pipe(text, disable=["tagger"]):

  if doc.ents:
    
    k1 = ""  #ASP
    k2 = ""  #OPI

    for e in doc.ents:

      if (e.label_ == 'OPI'):
        e2 = e.text
        if k2 != "":
          k2+= ' , '+ e2
        else:
          k2 += e2  


      elif  (e.label == 'ASP'):
        e1 = e.text
        if k1 != "":
          k1+= ' , '+ e1
        else:
          k1 += e1 
         
    d1 = {'sentences': str(doc) , 'ASP':k1, 'OPI': k2}
    b = b.append(d1, ignore_index = True)

  else:

    d1 = {'sentences': str(doc)}
    b = b.append(d1, ignore_index = True)

        

In [91]:
b

,sentences,ASP,OPI
0,I got the IPhone 8 yesterday.\n,NaN,NaN
1,I am really happy with it!\n,,really happy
2,It looks and works like new so far.\n,,looks and works
3,Bought this to replace iPhone 5s.\n,NaN,NaN
4,I’m happy I did.\n,NaN,NaN
5,Only drawback I have found is has poor recepti...,,poor reception
6,"Works great, looks like new, so far no issues.\n",,Works great
7,Great price\n,,Great
8,This is the first time I’ve bought a phone not...,,"broken , debris , fully functioning"
